# OpenAI Agents with Couchbase via Model Context Protocol (MCP) - A Tutorial

This notebook demonstrates how to build an intelligent agent using [OpenAI agents framework](https://platform.openai.com/docs/guides/agents) that can interact with a Couchbase database. The key to this interaction is the Model Context Protocol (MCP), which allows the AI agent to seamlessly connect to and use Couchbase as a tool.

## What is the Model Context Protocol (MCP)?

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) is an open standard designed to standardize how AI assistants and applications connect to and interact with external data sources, tools, and systems. Think of MCP as a universal adapter that allows AI models to seamlessly access the context they need to produce more relevant, accurate, and actionable responses.

**Key Goals and Features of MCP:**

*   **Standardized Communication:** MCP provides a common language and structure for AI models to communicate with diverse backend systems, replacing the need for numerous custom integrations.
*   **Enhanced Context Management:** It helps manage the limited context windows of LLMs efficiently, enabling them to maintain longer, more coherent interactions and leverage historical data.
*   **Secure Data Access:** MCP emphasizes secure connections, allowing developers to expose data through MCP servers while maintaining control over their infrastructure.
*   **Tool Use and Actionability:** It enables LLMs to not just retrieve information but also to use external tools and trigger actions in other systems.
*   **Interoperability:** Fosters an ecosystem where different AI tools, models, and data sources can work together more cohesively.

MCP aims to break down data silos, making it easier for AI to integrate with real-world applications and enterprise systems, leading to more powerful and context-aware AI solutions.

**MCP Typically Follows a Client-Server Architecture:**
*   **MCP Hosts/Clients:** Applications (like AI assistants, IDEs, or other AI-powered tools) that want to access data or capabilities. In this demo, this notebook, through LangChain, acts as an MCP client.
*   **MCP Servers:** Lightweight programs that expose specific data sources or tools (e.g., a database, an API) through the standardized MCP. The `mcp-server-couchbase` project fulfills this role for Couchbase.



# Before you start
## Get Credentials for OpenAI
Please follow the [instructions](https://platform.openai.com/docs/quickstart) to generate the OpenAI credentials.
## Create and Deploy Your Free Tier Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy a forever free tier operational cluster. This account provides you with an environment where you can explore and learn about Capella with no time constraint.

To learn more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html).

### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met.

* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the required bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.
* Your Capella free-tier account includes a travel-sample bucket, with sample documents used for booking and travel purposes. You can find more information [here](https://docs.couchbase.com/cloud/get-started/run-first-queries.html).

## Setup Instructions

Before running this notebook, ensure you have the following prerequisites met:

*   **`mcp-server-couchbase` Project:**
    *   Clone the `mcp-server-couchbase` project from [here](https://github.com/Couchbase-Ecosystem/mcp-server-couchbase).
    *   You'll need the local path to this project to start the MCP server from this notebook. **Update this path in the `StdioServerParameters` cell later if yours is different.**
*   **Set Environment Variables:** This notebook loads the OpenAI API key and other environment variables from the `.env` file. Include the following:
    ```env
    OPENAI_API_KEY=your_openai_api_key_here
    CB_CONNECTION_STRING=your_couchbase_connection_string
    CB_USERNAME=your_couchbase_username
    CB_PASSWORD=your_couchbase_password
    CB_BUCKET_NAME=your_target_bucket # e.g., travel-sample
    ```
    We have already included a `.env.sample` file. Change the file name to `.env` and fill in the environment variables.
*   **Setup uv:** uv is a modern and fast python package and project manager. We will use uv to run the MCP server. Install uv from [here](https://docs.astral.sh/uv/getting-started/installation/#installing-uv).
*   **Python Libraries:** Install the necessary libraries by running the code cell below.

In [1]:
!pip install -q 'openai-agents==0.0.16' 'python-dotenv==1.1.0'

## Importing Necessary Libraries

This cell imports the essential Python tools for our project:

*   **`dotenv`**: For loading secret API keys and other settings from a `.env` file.
*   **`agents` (`Agent`, `Runner`, `gen_trace_id`, `trace`)**: Custom modules likely related to the agent framework being used, providing core agent functionalities, a way to run agents, generate trace IDs for debugging/monitoring, and tracing capabilities.
*   **`agents.mcp` (`MCPServer`, `MCPServerStdio`)**: Custom modules for interacting with the Model Context Protocol (MCP), specifically for setting up an MCP server, possibly one that communicates over standard input/output (stdio).

Running this cell makes all these components ready to use.

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, gen_trace_id, trace
from agents.mcp import MCPServer, MCPServerStdio

load_dotenv()

True

## Defining the Question-Answering Function

This cell defines an asynchronous function `qna(agent)` that we'll use to interact with our agent.

*   It takes the created `agent` as an argument.
*   The function defines a series of example questions (as `message` strings) that are posed to the agent.
*   For each question, it calls `await Runner.run(starting_agent=agent, input=message)` to get the agent's response. The `Runner.run` method is responsible for executing the agent with the given input.
*   The agent, in turn, queries the Couchbase MCP to retrieve travel-related data, formats it, and presents it as the final output, which is then printed.
*   This function allows us to easily test the agent with multiple queries in sequence and observe its responses.

In [3]:
async def qna(agent):
    message = "Tell me about the database that you are connected to."
    print(f"\n\n**Running**: {message}\n")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)
    print('-'*50)

    message = "List out the top 5 hotels by the highest aggregate rating?"
    print(f"\n\n**Running**: {message}\n")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)
    print('-'*50)

    message = "Recommend me a flight and hotel from New York to San Francisco"
    print(f"\n\n**Running**: {message}\n")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)
    print('-'*50)

    message = "I'm going to the UK for 1 week. Recommend some great spots to visit for sightseeing. Also mention the respective prices of those places for adults and kids."
    print(f"\n\n**Running**: {message}\n")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)
    print('-'*50)

    message = "My budget is around 30 pounds a night. What will be the best hotel to stay in?"
    print(f"\n\n**Running**: {message}\n")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)
    print('-'*50)

## Defining the System Prompt

The system prompt is a crucial piece of instruction given to the Large Language Model (LLM) that powers our agent. It sets the context, defines the agent's persona, capabilities, and constraints.

In this system prompt:
*   We explain the **Couchbase data hierarchy** (Cluster, Bucket, Scope, Collection, Document) to help the LLM understand how the data is organized.
*   We specifically instruct the agent that **"The data is inside `inventory` scope, so use only that scope."** This focuses the agent on the relevant part of the `travel-sample` database.
*   We provide **SQL++ query generation guidelines**:
    *   "Any query you generate needs to have only the collection name in the FROM clause."
    *   "Every field, collection, scope or bucket name inside the query should be inside backticks."
*   The overall goal is to guide the LLM to use the provided MCP tools (which will be Couchbase operations) effectively and to formulate correct SQL++ queries for the `inventory` scope.

A well-crafted system prompt significantly improves the agent's performance and reliability.


In [4]:
system_prompt = """Couchbase organizes data with the following hierarchy (from top to bottom):

                1. Cluster

                The overall container of all Couchbase data and services.

                2. Bucket

                A bucket is similar to a database in traditional systems.

                Each bucket contains multiple scopes.

                Example: "users", "analytics", "products"

                3. Scope

                A scope is a namespace within a bucket that groups collections.

                Scopes help isolate data for different microservices or tenants.

                Default scope name: _default

                4. Collection

                The equivalent of a table in relational databases.

                Collections store JSON documents.

                Default collection name: _default

                5. Document

                The atomic data unit (usually JSON) stored in a collection.

                Each document has a unique key within its collection.

                Use the tools to read the database answer questions based on this database.
                The data is inside `inventory` scope, so use only that scope.
                Any query you generate needs to have only the collection name in the FROM clause.
                Every field, collection, scope or bucket name inside the query should be inside backticks."""


## Configuring the Agent and Language Model

This cell sets up the agent.

*   The code defines an asynchronous function `run` that initializes an AI agent.
*   This Assistant is configured with specific instructions (from `system_prompt`), uses the "gpt-4.1" model, and is connected to the Couchbase MCP server (`mcp_server`).
*   The function then calls `qna`, to start the question-and-answer process with the database.


In [5]:
async def run(mcp_server: MCPServer):
    agent = Agent(
        name="Assistant",
        instructions=system_prompt,
        model="gpt-4.1",
        mcp_servers=[mcp_server],
    )

    await qna(agent=agent)

## Initialize the MCP Server and Running The Agent


This cell:

*   **Starts Couchbase MCP Server:** Initializes the `mcp-server-couchbase` using `MCPServerStdio`.
*   **Runs as Background Process:** The server script (`src/mcp_server.py`) is executed using `uv` (Python project manager).
*   **Uses `.env` for Credentials:** The `--env-file` argument specifies the path to a file containing database connection details.
*   **Enables Agent-DB Communication:** The started `server` object is passed to the `run` function, allowing the AI agent to interact with Couchbase via MCP.
*   **Observability:** Sets up tracing to monitor agent interactions.

The agent will then attempt to answer the series of questions defined in the `qna` function by:
*   Reasoning about the question.
*   Deciding if a Couchbase tool is needed.
*   Formulating a SQL++ query (if appropriate, based on the system prompt).
*   Executing the tool (which sends the query to the MCP server, which then runs it on Couchbase).
*   Using the tool's output to generate a natural language response.

**IMPORTANT:** You **MUST** update the `"--directory"` and `"--env-file"`, path to point to the correct location of your cloned `mcp-server-couchbase` repository.

In [6]:
async with MCPServerStdio(
    params={
        "command": "uv",
        "args": [
            "--directory", 
            "/path/to/mcp-server-couchbase",
            "run",
            "--env-file", "/path/to/.env",
            "src/mcp_server.py"
        ],
    }
) as server:
    trace_id = gen_trace_id()
    with trace(trace_id=trace_id, workflow_name="CouchbaseDemo"):
        print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}\n")
        print("-"*25, "Starting Run", "-"*25)
        await run(server)

View trace: https://platform.openai.com/traces/trace?trace_id=trace_11767c0e64764bc08c4fe5ba1ef88192

------------------------- Starting Run -------------------------


**Running**: Tell me about the database that you are connected to.

I am connected to a Couchbase database that contains several scopes and collections. Here’s an overview of the main data of interest according to your instructions:

- The primary scope we focus on is inventory.
- Within the inventory scope, there are six collections:
    - cache
    - route
    - landmark
    - hotel
    - airport
    - airline

Each collection stores JSON documents related to its category. If you'd like specifics about any collection (such as its data schema or example documents), let me know which one you're interested in!
--------------------------------------------------


**Running**: List out the top 5 hotels by the highest aggregate rating?

The top 5 hotels by highest aggregate (average) overall rating are:

1. I Need A Holiday

Errored MCP tool result: meta=None content=[] isError=False


Here are recommendations for your trip from New York to San Francisco:

Hotel in San Francisco:
- Name: Inn on Castro
- Address: 321 Castro St, San Francisco, CA, United States
- Description: An upscale bed and breakfast in a restored house; offers free internet and parking, allows pets.
- Price Range: $95–$190
- Contact: +1 415 861-0321 | Innkeeper@innoncastro.com
- Website: http://www.innoncastro.com/

No flight could be retrieved at this time, but I can attempt to help further if you want specific airline info or more travel options! Let me know if you need another hotel or want to try again for flight details.
--------------------------------------------------


**Running**: I'm going to the UK for 1 week. Recommend some great spots to visit for sightseeing. Also mention the respective prices of those places for adults and kids.

Here are some great sightseeing spots you can visit in the UK for 1 week. I've included available pricing information for adults and kids where possible:


Errored MCP tool result: meta=None content=[] isError=False


Here are some of the best hotels you can stay at in the UK within or below your budget of around £30 per night, listing the most attractive options:

### 1. Glencoe Youth Hostel
- **City:** Highland
- **Country:** United Kingdom
- **Description:** 62 Family Bunk Rooms.
- **Price:** £17.00 per night
- **Perfect for:** Travelers who don't mind hostel accommodations and want to save money.

### 2. Rhossili Bunkhouse
- **City:** Swansea
- **Country:** United Kingdom
- **Description:** Within easy walking distance of three beaches, great for groups or families.
- **Price:** From £15 per person per night
- **Note:** Excellent for beach lovers or group travel.

### 3. The Greenhead Hotel and Hostel
- **City:** Greenhead
- **Country:** United Kingdom
- **Description:** Short distance from the historic wall, offers both hotel and hostel experiences.
- **Price:** Hostel (£15 pppn), Hotel (£50–£80 pppn)
- **Recommendation:** Hostel option for budget.

### 4. Once Brewed YHA Hostel
- **City:** Nor